In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby

In [2]:
df = pd.read_csv('../data/Iris/Iris-35-attcon.csv')

In [3]:
appr_df = df.copy() #appr_df will only be used for building case lists while finding approximations

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,-,-,Iris-setosa
1,-,3,-,-,Iris-setosa
2,4.7,3.2,1.3,-,Iris-setosa
3,4.6,-,1.5,-,Iris-setosa
4,-,-,1.4,0.2,Iris-setosa
5,-,-,-,0.4,Iris-setosa
6,4.6,3.4,1.4,-,Iris-setosa
7,-,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,-,Iris-setosa


### Handling Missing attributes using attribute-concept values !!

#### Defining Goal Set !!

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'class'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]


#### Building Case List !!

In [9]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [10]:
case_list = []

In [11]:
def discretize(numeric_col):
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '-' in point_list:
        point_list.remove('-')
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),2))
        
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [12]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

In [13]:
case_list

['sepal_length,4.3..4.35',
 'sepal_length,4.35..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.65',
 'sepal_length,4.65..7.9',
 'sepal_length,4.3..4.75',
 'sepal_length,4.75..7.9',
 'sepal_length,4.3..4.85',
 'sepal_length,4.85..7.9',
 'sepal_length,4.3..4.95',
 'sepal_length,4.95..7.9',
 'sepal_length,4.3..5.05',
 'sepal_length,5.05..7.9',
 'sepal_length,4.3..5.15',
 'sepal_length,5.15..7.9',
 'sepal_length,4.3..5.25',
 'sepal_length,5.25..7.9',
 'sepal_length,4.3..5.35',
 'sepal_length,5.35..7.9',
 'sepal_length,4.3..5.45',
 'sepal_length,5.45..7.9',
 'sepal_length,4.3..5.55',
 'sepal_length,5.55..7.9',
 'sepal_length,4.3..5.65',
 'sepal_length,5.65..7.9',
 'sepal_length,4.3..5.75',
 'sepal_length,5.75..7.9',
 'sepal_length,4.3..5.85',
 'sepal_length,5.85..7.9',
 'sepal_length,4.3..5.95',
 'sepal_length,5.95..7.9',
 'sepal_length,4.3..6.05',
 'sepal_length,6.05..7.9',
 'sepal_length,4.3..6.15',
 'sepal_length,6.15..7.9',
 'sepal_length,4.3..6.25',
 's

#### Fill up missing values !!

In [14]:
df2 = df.copy()

In [15]:
for col in df:
    for i, row_value in df[col].iteritems():
        
        #This is for attribute-concept value interpretation
        #For Lost value interpretation, we don't add that case to any block at all
        if row_value == '-':
            
            final_values = []
            print(col,i)
            concept_tmp = df[concept][i]
            print(concept_tmp)
            
            for item_list in goal_list:
                if i in item_list:
                    final_values = item_list
                    
            print (final_values)
            
            #Taking out all unique cases corresponding to the matched concept
            col_values = []
            for item in final_values:
                col_values.append(df[col][item])
        
            
            uniqueList = []
            for letter in col_values:
                if letter not in uniqueList:
                    uniqueList.append(letter)
            
            if '-' in uniqueList:
                uniqueList.remove('-')
                
            if '?' in uniqueList:
                uniqueList.remove('?')
            print(uniqueList)
            
#             print(df.loc[i,col])
            df2.at[i, col] = uniqueList
            
    print("\n")

sepal_length 1
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 4
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 5
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 7
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
['3.3', '2.7', '2.9', '3', '2.5', '3.6', '3.2', '3.8', '2.6', '2.2', '2.8', '3.4', '3.1']
sepal_width 123
Iris-viginica
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
['3.3', '2.7', '2.9', '3', '2.5', '3.6', '3.2', '3.8', '2.6', '2.2', '2.8', '3.4', '3.1']
sepal_width 124
Iris-viginica
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
['3.3',

['6', '5.1', '5.9', '5.6', '6.6', '4.5', '6.3', '5.8', '5.3', '6.7', '5', '5.7', '4.9', '6.1', '6.4', '5.5', '4.8', '5.2']
petal_length 149
Iris-viginica
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
['6', '5.1', '5.9', '5.6', '6.6', '4.5', '6.3', '5.8', '5.3', '6.7', '5', '5.7', '4.9', '6.1', '6.4', '5.5', '4.8', '5.2']


petal_width 0
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['0.2', '0.4', '0.1', '0.3', '0.5', '0.6']
petal_width 1
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4

sort_col 50
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '1', '1.1', '1.2', '1.3']
sort_col 51
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '1', '1.1', '1.2', '1.3']
sort_col 52
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '1', '1.1', '1.2', '1.3']
sort_col 53
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 6

In [16]:
df2

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa,[]
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa,[]
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa,[]
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]


In [17]:
appr_df_attcon = df2.copy()

In [18]:
df = df2.copy()

In [19]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [20]:
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [21]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [22]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]]


In [23]:
df

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa,[]
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa,[]
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa,[]
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]


#### Building blocks for attribute-value pair !!

In [24]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == b:
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == b:
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [25]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       sepal_length,4.3..4.35[2, 34, 35, 5, 6, 36, 8, 39, 41, 42, 12, 43, 14, 45, 16, 47, 19, 23, 27, 29]
1       sepal_length,4.35..7.9[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
2       sepal_length,4.3..4.5[2, 5, 6, 8, 9, 12, 14, 16, 19, 23, 27, 29, 34, 35, 36, 39, 41, 42, 43, 45, 47]
3       sepal_length,4.5..7.9[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2

In [26]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [27]:
df2 = pd.DataFrame(data)

In [28]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"sepal_length,4.3..4.35","[2, 34, 35, 5, 6, 36, 8, 39, 41, 42, 12, 43, 1..."
1,"sepal_length,4.35..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
2,"sepal_length,4.3..4.5","[2, 5, 6, 8, 9, 12, 14, 16, 19, 23, 27, 29, 34..."
3,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
4,"sepal_length,4.3..4.65","[2, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 27, ..."
5,"sepal_length,4.65..7.9","[1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17,..."
6,"sepal_length,4.3..4.75","[2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 2..."
7,"sepal_length,4.75..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."
8,"sepal_length,4.3..4.85","[2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 19, 2..."
9,"sepal_length,4.85..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 15, 16, 17, 18, 19..."


### Start of Lower/Upper & Probabilistic Approximations !!

#### Building Chacteristic Set !!

In [29]:
attributes = list(appr_df)
del attributes[-1]

In [30]:
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [31]:
appr_df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,-,-,Iris-setosa
1,-,3,-,-,Iris-setosa
2,4.7,3.2,1.3,-,Iris-setosa
3,4.6,-,1.5,-,Iris-setosa
4,-,-,1.4,0.2,Iris-setosa
5,-,-,-,0.4,Iris-setosa
6,4.6,3.4,1.4,-,Iris-setosa
7,-,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,-,Iris-setosa


In [32]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(appr_df[item]):
        temp = appr_df[item].unique()
        print(temp)
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

sepal_length
['5.1' '-' '4.7' '4.6' '4.4' '4.9' '5.4' '4.8' '4.3' '5.8' '5' '5.2' '5.5'
 '5.3' '6.9' '6.5' '5.7' '6.6' '5.9' '6.7' '5.6' '6.2' '6.1' '6.8' '6'
 '6.3' '7.1' '7.3' '7.2' '7.7' '6.4' '7.9']
sepal_width
['3.5' '3' '3.2' '-' '3.4' '2.9' '3.1' '3.7' '3.8' '3.6' '3.3' '4.1' '2.3'
 '2.8' '2.4' '2.7' '2.2' '2.6' '2.5']
petal_length
['-' '1.3' '1.5' '1.4' '1.2' '1.7' '1' '1.9' '1.6' '4.7' '4.9' '4' '4.6'
 '4.5' '3.3' '3.9' '3.5' '4.2' '3.6' '4.1' '4.8' '4.3' '4.4' '5' '3.8'
 '3.7' '3' '6' '5.1' '5.9' '5.6' '6.6' '6.3' '5.8' '5.3' '6.7' '5.7' '6.1'
 '6.4' '5.5' '5.2']
petal_width
['-' '0.2' '0.4' '0.1' '0.3' '0.5' '0.6' '1.5' '1.3' '1' '1.4' '1.1' '1.8'
 '1.2' '1.6' '2.5' '1.9' '2.1' '2.2' '1.7' '2' '2.3' '2.4']


In [33]:
case_list

['sepal_length,5.1',
 'sepal_length,4.7',
 'sepal_length,4.6',
 'sepal_length,4.4',
 'sepal_length,4.9',
 'sepal_length,5.4',
 'sepal_length,4.8',
 'sepal_length,4.3',
 'sepal_length,5.8',
 'sepal_length,5',
 'sepal_length,5.2',
 'sepal_length,5.5',
 'sepal_length,5.3',
 'sepal_length,6.9',
 'sepal_length,6.5',
 'sepal_length,5.7',
 'sepal_length,6.6',
 'sepal_length,5.9',
 'sepal_length,6.7',
 'sepal_length,5.6',
 'sepal_length,6.2',
 'sepal_length,6.1',
 'sepal_length,6.8',
 'sepal_length,6',
 'sepal_length,6.3',
 'sepal_length,7.1',
 'sepal_length,7.3',
 'sepal_length,7.2',
 'sepal_length,7.7',
 'sepal_length,6.4',
 'sepal_length,7.9',
 'sepal_width,3.5',
 'sepal_width,3',
 'sepal_width,3.2',
 'sepal_width,3.4',
 'sepal_width,2.9',
 'sepal_width,3.1',
 'sepal_width,3.7',
 'sepal_width,3.8',
 'sepal_width,3.6',
 'sepal_width,3.3',
 'sepal_width,4.1',
 'sepal_width,2.3',
 'sepal_width,2.8',
 'sepal_width,2.4',
 'sepal_width,2.7',
 'sepal_width,2.2',
 'sepal_width,2.6',
 'sepal_width,2

In [34]:
appr_df_attcon

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa,[]
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa,[]
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa,[]
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]


In [35]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in appr_df_attcon.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in appr_df_attcon.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == float(b):
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == float(b):
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [36]:
df = df.drop(columns=['sort_col'])

In [37]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa


In [38]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [39]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'sepal_length,5.1': [1,
  2,
  5,
  6,
  8,
  12,
  16,
  18,
  19,
  20,
  22,
  23,
  24,
  27,
  29,
  34,
  35,
  36,
  39,
  40,
  41,
  42,
  43,
  45,
  47,
  51,
  52,
  57,
  61,
  63,
  64,
  65,
  70,
  73,
  74,
  75,
  76,
  79,
  81,
  88,
  90,
  91,
  93,
  95,
  96,
  99],
 'sepal_length,4.7': [2,
  3,
  5,
  6,
  8,
  12,
  16,
  19,
  23,
  27,
  29,
  30,
  34,
  35,
  36,
  39,
  41,
  42,
  43,
  45,
  47],
 'sepal_length,4.6': [2,
  4,
  5,
  6,
  7,
  8,
  12,
  16,
  19,
  23,
  27,
  29,
  34,
  35,
  36,
  39,
  41,
  42,
  43,
  45,
  47,
  48],
 'sepal_length,4.4': [2,
  34,
  35,
  5,
  6,
  36,
  8,
  9,
  39,
  41,
  12,
  42,
  43,
  45,
  16,
  47,
  19,
  23,
  27,
  29],
 'sepal_length,4.9': [2,
  131,
  5,
  6,
  133,
  8,
  10,
  138,
  12,
  139,
  142,
  143,
  16,
  145,
  19,
  23,
  27,
  29,
  34,
  35,
  36,
  38,
  39,
  41,
  42,
  43,
  45,
  47,
  51,
  52,
  57,
  58,
  61,
  63,
  64,
  65,
  70,
  73,
  74,
  75,
  76,
  79,
  81,
  

In [40]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [41]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + df.loc[index,cols]
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,3.5']
When values are list
['petal_length,1.3']
['petal_length,1.3', 'petal_length,1.5']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1', 'petal_length,1.9']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1', 'petal_length,1.9', 'petal_length,1.6']
Union Set:  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38

['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width,3.1', 'sepal_width,3.7', 'sepal_width,3.8', 'sepal_width,3.6', 'sepal_width,3.3']
['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width,3.1', 'sepal_width,3.7', 'sepal_width,3.8', 'sepal_width,3.6', 'sepal_width,3.3', 'sepal_width,4.1']
['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width,3.1', 'sepal_width,3.7', 'sepal_width,3.8', 'sepal_width,3.6', 'sepal_width,3.3', 'sepal_width,4.1', 'sepal_width,2.3']
Union Set:  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 59, 61, 62, 64, 65, 66, 67, 68, 70, 71, 73, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 92, 94, 96, 97, 98, 99, 101, 103, 104, 105, 106, 108

{1, 2, 3, 35, 37, 6, 39, 41, 10, 42, 12, 43, 14, 45, 16, 17, 21, 22, 30, 31}
When item is found as key in the block
{1, 2, 3, 37, 39, 10, 42, 12, 43, 21, 30, 31}
Final tmp_set:  {1, 2, 3, 37, 39, 10, 42, 12, 43, 21, 30, 31}
This is the final value:  {1, 2, 3, 37, 39, 10, 42, 12, 43, 21, 30, 31}
K_43


When value is single
['sepal_length,5']
When value is single
['sepal_length,5', 'sepal_width,3.5']
When value is single
['sepal_length,5', 'sepal_width,3.5', 'petal_length,1.6']
When value is single
['sepal_length,5', 'sepal_width,3.5', 'petal_length,1.6', 'petal_width,0.6']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{2, 5, 6, 8, 12, 16, 19, 23, 26, 27, 29, 34, 35, 36, 39, 41, 42, 43, 44, 45, 47, 50, 51, 52, 57, 61, 63, 64, 65, 70, 73, 74, 75, 76, 79, 81, 88, 90, 91, 93, 94, 95, 96}
When item is found as key in the block
{34, 5, 6, 41, 43, 44, 45, 47, 16, 50}
When item is found as key in the block
{6, 44, 45, 47, 16}
When item is found as key in the block

When values are list
['sepal_width,3.2']
['sepal_width,3.2', 'sepal_width,3.1']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9', 'sepal_width,2.6']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_wid

Final tmp_set:  {130, 67, 98, 69, 134, 100, 79, 114, 52, 149, 150, 84, 85, 87, 117}
This is the final value:  {98, 67, 100, 69, 79, 114, 52, 149, 150, 84, 85, 87, 117}
K_79


When value is single
['sepal_length,5.7']
When value is single
['sepal_length,5.7', 'sepal_width,2.6']
When values are list
['petal_length,4.7']
['petal_length,4.7', 'petal_length,4.9']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6', 'petal_length,4.5']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6', 'petal_length,4.5', 'petal_length,3.3']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6', 'petal_length,4.5', 'petal_length,3.3', 'petal_length,3.9']
['petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6', 'petal_length,4.5', 'petal_length,3.3', 'petal_length,3.9', 'peta

{101}
Final tmp_set:  {101}
This is the final value:  {101}
K_101


When value is single
['sepal_length,5.8']
When value is single
['sepal_length,5.8', 'sepal_width,2.7']
When value is single
['sepal_length,5.8', 'sepal_width,2.7', 'petal_length,5.1']
When value is single
['sepal_length,5.8', 'sepal_width,2.7', 'petal_length,5.1', 'petal_width,1.9']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{2, 131, 5, 6, 133, 8, 138, 139, 12, 142, 15, 16, 143, 145, 19, 23, 27, 29, 34, 35, 36, 39, 41, 42, 43, 45, 47, 51, 52, 57, 61, 63, 64, 65, 68, 70, 73, 74, 75, 76, 79, 81, 83, 88, 90, 91, 93, 95, 96, 102, 106, 109, 111, 112, 114, 115, 116, 124, 125}
When item is found as key in the block
{142, 143, 145, 52, 57, 61, 68, 70, 73, 76, 79, 81, 83, 88, 90, 95, 96, 102, 112, 115, 116, 124, 125}
When item is found as key in the block
{102, 142, 143, 112, 115, 124}
When item is found as key in the block
{112, 115, 102, 143}
Final tmp_set:  {112, 115, 102, 143}
This is the f

Final Union Set:  [52, 53, 56, 59, 66, 67, 69, 71, 73, 77, 78, 79, 80, 84, 85, 86, 87, 89, 93, 94, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
When item is found as key in the block
{131, 133, 138, 139, 142, 143, 144, 145, 51, 52, 57, 61, 63, 64, 65, 70, 73, 74, 75, 76, 77, 79, 81, 88, 90, 91, 93, 95, 96, 106, 109, 111, 112, 113, 114, 115, 116, 124, 125}
When item is found as key in the block
{139, 142, 145, 52, 57, 61, 70, 73, 76, 77, 79, 81, 88, 90, 96, 106, 112, 113, 115, 116, 124, 125}
When item is found as key in the block
{106, 145, 113, 115, 125}
Final tmp_set:  {106, 145, 113, 115, 125}
This is the final value:  {106, 145, 113, 115, 125}
K_113


When values are list
['sepal_length,6.3']
['sepal_length,6.3', 'sepal_length,5.8']
['sepal_length,6.3', 'sepal_length,5

When item is found as key in the block
{128, 131, 133, 135, 138, 139, 142, 143, 145, 51, 52, 57, 61, 63, 64, 65, 70, 72, 73, 74, 75, 76, 79, 81, 88, 90, 91, 92, 93, 95, 96, 106, 109, 111, 112, 114, 115, 116, 124, 125}
When item is found as key in the block
{135, 142, 145, 52, 57, 61, 70, 73, 76, 79, 81, 88, 90, 91, 93, 96, 112, 115, 116, 124, 125}
When item is found as key in the block
{112, 124, 135}
When item is found as key in the block
{135}
Final tmp_set:  {135}
This is the final value:  {135}
K_135


When value is single
['sepal_length,7.7']
When value is single
['sepal_length,7.7', 'sepal_width,3']
When value is single
['sepal_length,7.7', 'sepal_width,3', 'petal_length,6.1']
When value is single
['sepal_length,7.7', 'sepal_width,3', 'petal_length,6.1', 'petal_width,2.3']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{131, 133, 136, 106, 138, 139, 109, 142, 111, 112, 143, 114, 115, 116, 145, 118, 119, 123, 124, 125}
When item is found as key in the

['sepal_length,6.5', 'sepal_width,3', 'petal_length,5.2']
When value is single
['sepal_length,6.5', 'sepal_width,3', 'petal_length,5.2', 'petal_width,2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{131, 133, 138, 139, 142, 143, 145, 148, 51, 52, 55, 57, 61, 63, 64, 65, 70, 73, 74, 75, 76, 79, 81, 88, 90, 91, 93, 95, 96, 105, 106, 109, 111, 112, 114, 115, 116, 117, 124, 125}
When item is found as key in the block
{139, 142, 145, 148, 52, 57, 61, 70, 73, 76, 79, 81, 88, 90, 96, 105, 106, 112, 115, 116, 117, 124, 125}
When item is found as key in the block
{105, 112, 148, 117, 124}
When item is found as key in the block
{148, 117}
Final tmp_set:  {148, 117}
This is the final value:  {148, 117}
K_148


When value is single
['sepal_length,6.2']
When value is single
['sepal_length,6.2', 'sepal_width,3.4']
When values are list
['petal_length,6']
['petal_length,6', 'petal_length,5.1']
['petal_length,6', 'petal_length,5.1', 'petal_length,5.9']
['petal_length,6'

In [42]:
dic

{'K_1': {1, 5, 6, 16, 18, 34, 40, 41, 43, 45, 47},
 'K_2': {2,
  4,
  5,
  6,
  13,
  14,
  15,
  16,
  17,
  21,
  26,
  28,
  32,
  34,
  37,
  38,
  39,
  40,
  43,
  45,
  46,
  47,
  49,
  50,
  52,
  54,
  57,
  61,
  68,
  70,
  73,
  76,
  79,
  81,
  82,
  83,
  85,
  88,
  90,
  96,
  99,
  106,
  112,
  115,
  116,
  124,
  125,
  139,
  142,
  145},
 'K_3': {3, 6, 16, 30, 43, 45},
 'K_4': {2, 4, 6, 8, 12, 16, 35, 39, 42, 45},
 'K_5': {1,
  2,
  5,
  7,
  9,
  10,
  12,
  13,
  18,
  21,
  29,
  30,
  31,
  34,
  37,
  39,
  42,
  48,
  50},
 'K_6': {1,
  2,
  3,
  4,
  6,
  7,
  9,
  10,
  13,
  16,
  17,
  18,
  20,
  22,
  27,
  29,
  30,
  32,
  33,
  34,
  36,
  37,
  42,
  45,
  48,
  49},
 'K_7': {5, 6, 7, 12, 16, 29, 34, 45},
 'K_8': {4, 8, 12, 21, 28, 32, 37, 40, 49},
 'K_9': {5, 6, 9, 16, 34, 45},
 'K_10': {5, 6, 10, 16, 34, 35, 38, 43, 45, 47},
 'K_11': {11, 21, 32},
 'K_12': {4,
  5,
  7,
  8,
  12,
  15,
  21,
  25,
  27,
  28,
  29,
  32,
  34,
  37,
  40,
  43

#### Lower Approximation !!

In [43]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [44]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [45]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}, {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}, {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127}]


#### Upper Approximation !!

In [46]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [47]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [48]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 57, 61, 64, 65, 68, 70, 73, 75, 76, 79, 81, 82, 83, 85, 88, 90, 94, 96, 99, 106, 112, 115, 116, 124, 125, 139, 142, 145}, {141, 143, 145, 149, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 106, 112, 114, 115, 117, 120, 124, 125}, {51, 52, 55, 57, 61, 62, 63, 64, 65, 67, 68, 70, 71, 73, 74, 75, 76, 78, 79, 81, 83, 84, 87, 88, 90, 91, 93, 95, 96, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}]


#### Middle Approximation !!

In [49]:
first_column = list(dic.keys())

In [50]:
second_column = list(dic.values())

In [51]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [52]:
prob_approx

,charset_name,charset_value
0,K_1,"{1, 34, 5, 6, 40, 41, 43, 45, 47, 16, 18}"
1,K_2,"{2, 4, 5, 6, 139, 13, 14, 15, 16, 17, 142, 145..."
2,K_3,"{3, 6, 43, 45, 16, 30}"
3,K_4,"{2, 35, 4, 6, 39, 8, 42, 12, 45, 16}"
4,K_5,"{1, 2, 34, 37, 5, 7, 39, 9, 10, 42, 12, 13, 48..."
5,K_6,"{1, 2, 3, 4, 6, 7, 9, 10, 13, 16, 17, 18, 20, ..."
6,K_7,"{34, 5, 6, 7, 12, 45, 16, 29}"
7,K_8,"{32, 4, 37, 8, 40, 12, 49, 21, 28}"
8,K_9,"{34, 5, 6, 9, 45, 16}"
9,K_10,"{34, 35, 5, 6, 38, 10, 43, 45, 47, 16}"


In [53]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [54]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    print(prob_approx)
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    
    return probapprox  

In [55]:
middle_approximations = {}
for item in goal_list:
    print(item)
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
    charset_name                                      charset_value  \
0            K_1          {1, 34, 5, 6, 40, 41, 43, 45, 47, 16, 18}   
1            K_2  {2, 4, 5, 6, 139, 13, 14, 15, 16, 17, 142, 145...   
2            K_3                             {3, 6, 43, 45, 16, 30}   
3            K_4               {2, 35, 4, 6, 39, 8, 42, 12, 45, 16}   
4            K_5  {1, 2, 34, 37, 5, 7, 39, 9, 10, 42, 12, 13, 48...   
5            K_6  {1, 2, 3, 4, 6, 7, 9, 10, 13, 16, 17, 18, 20, ...   
6            K_7                      {34, 5, 6, 7, 12, 45, 16, 29}   
7            K_8                 {32, 4, 37, 8, 40, 12, 49, 21, 28}   
8            K_9                              {34, 5, 6, 9, 45, 16}   
9           K_10             {34, 35, 5, 6, 38, 10, 43, 45, 47, 16}   
10          K_11           

    charset_name                                      charset_value  \
0            K_1          {1, 34, 5, 6, 40, 41, 43, 45, 47, 16, 18}   
1            K_2  {2, 4, 5, 6, 139, 13, 14, 15, 16, 17, 142, 145...   
2            K_3                             {3, 6, 43, 45, 16, 30}   
3            K_4               {2, 35, 4, 6, 39, 8, 42, 12, 45, 16}   
4            K_5  {1, 2, 34, 37, 5, 7, 39, 9, 10, 42, 12, 13, 48...   
5            K_6  {1, 2, 3, 4, 6, 7, 9, 10, 13, 16, 17, 18, 20, ...   
6            K_7                      {34, 5, 6, 7, 12, 45, 16, 29}   
7            K_8                 {32, 4, 37, 8, 40, 12, 49, 21, 28}   
8            K_9                              {34, 5, 6, 9, 45, 16}   
9           K_10             {34, 35, 5, 6, 38, 10, 43, 45, 47, 16}   
10          K_11                                       {32, 11, 21}   
11          K_12  {32, 34, 4, 37, 5, 7, 8, 40, 43, 12, 15, 47, 4...   
12          K_13                  {2, 34, 5, 6, 39, 13, 45, 46, 16}   
13    

[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
    charset_name                                      charset_value  \
0            K_1          {1, 34, 5, 6, 40, 41, 43, 45, 47, 16, 18}   
1            K_2  {2, 4, 5, 6, 139, 13, 14, 15, 16, 17, 142, 145...   
2            K_3                             {3, 6, 43, 45, 16, 30}   
3            K_4               {2, 35, 4, 6, 39, 8, 42, 12, 45, 16}   
4            K_5  {1, 2, 34, 37, 5, 7, 39, 9, 10, 42, 12, 13, 48...   
5            K_6  {1, 2, 3, 4, 6, 7, 9, 10, 13, 16, 17, 18, 20, ...   
6            K_7                      {34, 5, 6, 7, 12, 45, 16, 29}   
7            K_8                 {32, 4, 37, 8, 40, 12, 49, 21, 28}   
8            K_9                              {34, 5, 6, 9, 45, 16}   
9           K_10             {34, 35, 5

In [56]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 57, 61, 64, 65, 68, 70, 73, 75, 76, 79, 81, 82, 83, 85, 88, 90, 94, 96, 99}, {141, 143, 145, 149, 150, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 106, 112, 114, 115, 117, 120, 124, 125}, {51, 52, 55, 57, 61, 64, 67, 68, 71, 75, 78, 83, 84, 87, 90, 91, 95, 96, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}]


In [57]:
df3=df2

In [58]:
import pylab as pl

In [59]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [60]:
def updateGoalIntersect(goal):
    for index, row in df3.iterrows():
        if row['goal_intersect'] != set():
            row['goal_intersetct'] = set(row['att_val']).intersection(set(goal))

In [61]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [62]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [63]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [64]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        #Check if intersecting elements are subset of Goal
        if A.issubset(original_goal):
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - df3['goal_intersect'].loc[selected_case]
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + concept_curr + ")"
            rule_set.append(rule)
            
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)
            

        #If not a subset of current goal
        else:
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
            
            updateGoalIntersect(A.intersection(current_goal))
            selected_case = findCases(df3)
                        
    return rule_set

In [65]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal
End of goal


In [66]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  38.722 Sec


['(petal_width,0.1..1.55) & (petal_length,1.0..4.75) & (sepal_width,2.85..4.1) & (sepal_length,4.3..5.55) -> (class,Iris-setosa)', '(petal_length,1.0..4.15) & (petal_width,0.1..1.05) & (sepal_width,3.15..4.1) -> (class,Iris-setosa)']
['(petal_width,1.15..1.55) & (petal_length,1.0..4.95) & (sepal_length,4.3..6.25) -> (class,Iris-versicolor)', '(sepal_length,6.35..7.9) & (petal_width,0.1..1.55) & (sepal_length,4.3..7.0) -> (class,Iris-versicolor)']
['(sepal_length,5.75..7.9) & (petal_width,1.65..2.5) -> (class,Iris-viginica)', '(petal_length,4.95..6.7) & (petal_width,1.25..2.5) & (sepal_length,5.75..6.35) -> (class,Iris-viginica)']


In [67]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


In [68]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  47.343 Sec


['(petal_length,1.0..2.45) -> (class,Iris-setosa)']
['(petal_width,1.25..2.5) & (sepal_length,4.3..6.05) & (petal_width,0.1..1.35) -> (class,Iris-versicolor)', '(sepal_width,2.55..4.1) & (petal_width,0.1..1.55) & (sepal_width,2.2..3.05) & (petal_length,1.0..4.65) & (sepal_length,6.15..6.25) -> (class,Iris-versicolor)', '(sepal_length,6.55..7.9) & (sepal_width,3.05..4.1) & (petal_width,0.1..1.55) & (petal_length,1.0..4.95) -> (class,Iris-versicolor)']
['(sepal_width,2.2..3.35) & (petal_length,5.05..6.7) -> (class,Iris-viginica)', '(petal_length,4.75..6.7) & (petal_width,1.75..2.5) & (sepal_length,4.3..5.65) -> (class,Iris-viginica)', '(petal_length,4.75..6.7) & (petal_width,1.55..2.5) & (sepal_width,2.95..3.05) & (sepal_length,4.3..6.35) -> (class,Iris-viginica)']


In [69]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: MatplotlibDeprecationWarning: numpy.arange


In [70]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  43.297 Sec


['(sepal_width,2.85..4.1) & (petal_width,0.1..1.95) & (sepal_length,4.3..5.55) & (petal_length,1.0..5.4) -> (class,Iris-setosa)', '(sepal_length,4.3..4.95) & (sepal_width,2.95..3.05) -> (class,Iris-setosa)']
['(petal_width,1.15..1.55) & (petal_length,1.0..4.95) & (sepal_length,4.3..6.25) -> (class,Iris-versicolor)', '(sepal_length,6.35..7.9) & (petal_width,0.1..1.55) & (sepal_length,4.3..7.0) -> (class,Iris-versicolor)']
['(sepal_width,2.65..4.1) & (petal_width,1.45..2.5) & (sepal_length,6.05..6.85) -> (class,Iris-viginica)', '(petal_length,5.65..6.7) -> (class,Iris-viginica)']
